## Linear Regression

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import csv
import math
import pandas as pd
import random
from matplotlib import pyplot as plt

In [1]:
print('UBID:          hnitturk')
print('Person Number: 50291411')

UBID:          hnitturk
Person Number: 50291411


## Human Observed Data

## Fetch Data

In [ ]:
data_HOD = pd.read_csv('HumanObserved-Dataset/HumanObserved-Features-Data/HumanObserved-Features-Data.csv')
data_same = pd.read_csv('HumanObserved-Dataset/HumanObserved-Features-Data/same_pairs.csv');
data_diff = pd.read_csv('HumanObserved-Dataset/HumanObserved-Features-Data/diffn_pairs.csv');

## Creating Dictionary

In [ ]:
#create data dictionary to retrieve data into rawdata for processing based on key,value pairs
data_HOD = data_HOD.set_index('img_id',drop = True)
data_HOD=data_HOD.drop(["Unnamed: 0"], axis=1)
list_index_HOD = []
for i in list(data_HOD.index.values):
    list_index_HOD.append(i)
HOD_Dict = {}
for i in range(0,len(list_index_HOD)):
    HOD_Dict[list_index_HOD[i]]=list(data_HOD.loc[list_index_HOD[i]])

## Create raw data by concatenation

In [ ]:
data_same = data_same.drop(["target"], axis = 1)
list_same_imgNames = []
for i in data_same.index.values:
    list_same_imgNames.append(i)
same_data=[]
for i in range(len(data_same)):
    same_data.append(list(data_same.loc[list_same_imgNames[i]]))
final_same_data_concat = []
for i in same_data:
    temp = []
    for j in i:
        temp+=HOD_Dict.get(j)
    final_same_data_concat.append(temp)

data_diff = data_diff.drop(["target"], axis = 1)
list_diff_imgNames = []
for i in data_diff.index.values:
    list_diff_imgNames.append(i)
diff_data=[]

#keep same writer and different writer data in ratio 1:1
for i in range(0,791):
    diff_data.append(list(data_diff.loc[list_diff_imgNames[i]]))
temp_diff_data = []
final_diff_data_concat=[]
for i in diff_data:
    temp = []
    for j in i:
        temp+=HOD_Dict.get(j)
    temp_diff_data.append(temp)
    
for i in range(0,791):
    a = random.choice(temp_diff_data)
    final_diff_data_concat.append(a)

same_labels = [1]*len(final_same_data_concat)
diff_labels = [0]*len(final_diff_data_concat)
final_data_set_concat = []
final_data_set_concat.extend(final_same_data_concat)
final_data_set_concat.extend(final_diff_data_concat)
final_train_labels_concat = []
final_train_labels_concat.extend(same_labels)
final_train_labels_concat.extend(diff_labels)
temp = list(zip(final_data_set_concat,final_train_labels_concat))
for i in range(3):
    random.shuffle(temp)
d, t = zip(*temp)

temp_data1 = np.asarray(d)
RawData_concat = np.transpose(temp_data1)
RawTarget_concat = list(t)

## Create raw data by subtraction

In [ ]:
final_same_data=[]

# final_same_data = []
for i in same_data:
    temp1 = HOD_Dict.get(i[0])
    temp2 = HOD_Dict.get(i[1])
    final_same_data.append(np.absolute(np.asarray(temp1) - np.asarray(temp2)))

temp_diff_data = []
final_diff_data =[]
for i in diff_data:
    temp1 = HOD_Dict.get(i[0])
    temp2 = HOD_Dict.get(i[1]) 
    temp_diff_data.append(np.absolute(np.asarray(temp1) - np.asarray(temp2)))

for i in range(0,791):
    a = random.choice(temp_diff_data)
    final_diff_data.append(a)

In [ ]:
same_labels = [1]*len(final_same_data)
diff_labels = [0]*len(final_diff_data)
final_data_set = []
final_data_set.extend(final_same_data)
final_data_set.extend(final_diff_data)
final_train_labels= []
final_train_labels.extend(same_labels)
final_train_labels.extend(diff_labels)
temp = list(zip(final_data_set,final_train_labels))
for i in range(3):
    random.shuffle(temp)
d, t = zip(*temp)

temp_data1 = np.asarray(d)
RawData_diff = np.transpose(temp_data1)
RawTarget_diff = list(t)

In [ ]:
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10

C_Lambda = 0.03
M = 10
IsSynthetic = False

# The following functions is used to divide the target array to create seperate data partitions for training, 
# testing and validating.
def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    #print(str(TrainingPercent) + "% Training Target Generated..")
    return t

def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]
    #print(str(TrainingPercent) + "% Training Data Generated..")
    return d2

def GenerateValData(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Data Generated..")  
    return dataMatrix

def GenerateValTargetVector(rawData, ValPercent, TrainingCount):
    
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Target Data Generated..")
    return t

# BigSigma is a covariance matrix which is a diagonal matrix resulting from applying the Gaussian Radial Basis
# functions on the data
def GenerateBigSigma(Data, MuMatrix,TrainingPercent,IsSynthetic):
    BigSigma    = np.zeros((len(Data),len(Data)))
    DataT       = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))        
    varVect     = []
    for i in range(0,len(DataT[0])):
        vct = []
        for j in range(0,int(TrainingLen)):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct))
    
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]
    if IsSynthetic == True:
        BigSigma = np.dot(3,BigSigma)
    else:
        BigSigma = np.dot(200,BigSigma)
    ##print ("BigSigma Generated..")
    return BigSigma


def GetScalar(DataRow,MuRow, BigSigInv):  
    R = np.subtract(DataRow,MuRow)
    T = np.dot(BigSigInv,np.transpose(R))  
    L = np.dot(R,T)
    return L


# This expression returns the output of Gaussian radial basis function
def GetRadialBasisOut(DataRow,MuRow, BigSigInv):    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv))
    return phi_x

# This function generates the design matrix i which each row corresponds to the output value of the radial basis 
# functions for each input array of values for the training data. It is of the order training data rows by number 
# of radial basis functions.
def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    m = 0.0001
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 

    BigSigInv = np.linalg.inv(BigSigma + np.eye(BigSigma.shape[1])*m)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    #print ("PHI Generated..")
    return PHI

# This function returns the weights matrix for the training data for the closed form solution of the 
# linear regression problem. 
def GetWeightsClosedForm(PHI, T, Lambda):
    Lambda_I = np.identity(len(PHI[0]))
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    PHI_T       = np.transpose(PHI)
    PHI_SQR     = np.dot(PHI_T,PHI)
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)
    INTER       = np.dot(PHI_SQR_INV, PHI_T)
    W           = np.dot(INTER, T)
    ##print ("Training Weights Generated..")
    return W



def GetValTest(VAL_PHI,W):
    Y = np.dot(W,np.transpose(VAL_PHI))
    ##print ("Test Out Generated..")
    return Y

# This function defines the root mean square error function for evaluating accuracy. 
# This function should be minimized.
def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    ##print ("Accuracy Generated..")
    ##print ("Validation E_RMS : " + str(math.sqrt(sum/len(VAL_TEST_OUT))))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))

## Gradient Descent

In [ ]:
def gradient_descent(M, RawData, TrainingPercent, IsSythetic, TrainingTarget, C_Lambda, TestData, ValData, num_iter, lr):

    ErmsArr = []
    AccuracyArr = []
  
    
    # kmeans clustering is used to choose basis radial functions by clustering the data to create labels 
    # and then run radial basis functions to increase accuracy.

    kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingData))
    Mu = kmeans.cluster_centers_

    BigSigma     = GenerateBigSigma(RawData, Mu, TrainingPercent,IsSynthetic)
    TRAINING_PHI = GetPhiMatrix(RawData, Mu, BigSigma, TrainingPercent)
    W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda))
    TEST_PHI     = GetPhiMatrix(TestData, Mu, BigSigma, 100) 
    VAL_PHI      = GetPhiMatrix(ValData, Mu, BigSigma, 100)

    W_Now        = np.dot(220, W)
    La           = 2
    learningRate = lr
    L_Erms_Val   = []
    L_Erms_TR    = []
    L_Erms_Test  = []
    W_Mat        = []

    # Stochastic Gradient Descent also provides a solution to our problem by using a parameter known as learning rate 
    # which determine by howmuch shoud the weight matrix be updated for a more accurate prediction. The weight change 
    # that is added or subtracted should be minimum to achieve more accuracy.

    for i in range(0,num_iter):
        #print ('---------Iteration: ' + str(i) + '--------------')
        Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
        La_Delta_E_W  = np.dot(La,W_Now)
        Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
        Delta_W       = -np.dot(learningRate,Delta_E)
        W_T_Next      = W_Now + Delta_W
        W_Now         = W_T_Next

        #-----------------TrainingData Accuracy---------------------#
        TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next) 
        Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
        L_Erms_TR.append(float(Erms_TR.split(',')[1]))

        #-----------------ValidationData Accuracy---------------------#
        VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
        Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
        L_Erms_Val.append(float(Erms_Val.split(',')[1]))

        #-----------------TestingData Accuracy---------------------#
        TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
        Erms_Test = GetErms(TEST_OUT,TestDataAct)
        L_Erms_Test.append(float(Erms_Test.split(',')[1]))


    print ('----------Gradient Descent Solution--------------------')
    print ("M = "+str(M)+" \nLambda="+str(La)+"\nneta="+str(learningRate))
    print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
    print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
    print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))
   

## Data - Concantenation

## Same Writer

In [ ]:
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget_concat,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData_concat,TrainingPercent)

ValDataAct = np.array(GenerateValTargetVector(RawTarget_concat,ValidationPercent, (len(TrainingTarget))))
ValData    = GenerateValData(RawData_concat,ValidationPercent, (len(TrainingTarget)))

TestDataAct = np.array(GenerateValTargetVector(RawTarget_concat,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData_concat,TestPercent, (len(TrainingTarget)+len(ValDataAct)))


## Gradient Descent for concatenation of features

In [ ]:
gradient_descent(10, RawData_concat, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 100, 0.03)
gradient_descent(15, RawData_concat, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 100, 0.06)
gradient_descent(18, RawData_concat, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 100, 0.09)
gradient_descent(16, RawData_concat, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 100, 0.12)


## Different Writer

In [ ]:
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget_diff,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData_diff,TrainingPercent)

ValDataAct = np.array(GenerateValTargetVector(RawTarget_diff,ValidationPercent, (len(TrainingTarget))))
ValData    = GenerateValData(RawData_diff,ValidationPercent, (len(TrainingTarget)))

TestDataAct = np.array(GenerateValTargetVector(RawTarget_diff,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData_diff,TestPercent, (len(TrainingTarget)+len(ValDataAct)))

## Gradient Descent for difference of features

In [ ]:
gradient_descent(10, RawData_diff, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 100, 0.02)
gradient_descent(15, RawData_diff, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 100, 0.06)
gradient_descent(14, RawData_diff, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 100, 0.10)
gradient_descent(12, RawData_diff, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 100, 0.14)


## GSC Data

## Fetch Data

In [ ]:
data_GSC = pd.read_csv('GSC-Dataset/GSC-Features-Data/GSC-Features.csv')
data_same = pd.read_csv('GSC-Dataset/GSC-Features-Data/same_pairs.csv');
data_diff = pd.read_csv('GSC-Dataset/GSC-Features-Data/diffn_pairs.csv');

## Create Dictionary

In [ ]:
#create data dictionary to retrieve data into rawdata for processing based on key,value pairs
data_GSC = data_GSC.set_index('img_id',drop = True)
# data_GSC=data_GSC.drop(["Unnamed: 0"], axis=1)
list_index_GSC = []
for i in list(data_GSC.index.values):
    list_index_GSC.append(i)
GSC_Dict = {}
for i in range(0,len(list_index_GSC)):
    GSC_Dict[list_index_GSC[i]]=list(data_GSC.loc[list_index_GSC[i]])

## Create raw data by concatenation

In [ ]:
data_same = data_same.drop(["target"], axis = 1)
list_same_imgNames = []
for i in data_same.index.values:
    list_same_imgNames.append(i)
same_data=[]
for i in range(len(data_same)):
    same_data.append(list(data_same.loc[list_same_imgNames[i]]))
final_same_data_concat = []
for i in same_data:
    temp = []
    for j in i:
        temp+=GSC_Dict.get(j)
    final_same_data_concat.append(temp)

data_diff = data_diff.drop(["target"], axis = 1)
list_diff_imgNames = []
for i in data_diff.index.values:
    list_diff_imgNames.append(i)
diff_data=[]
for i in range(0,len(same_data)):
    diff_data.append(list(data_diff.loc[list_diff_imgNames[i]]))
temp_diff_data = []
final_diff_data_concat=[]
for i in diff_data:
    temp = []
    for j in i:
        temp+=GSC_Dict.get(j)
    temp_diff_data.append(temp)
    
    
#keep same writer and different writer data in ratio 1:1
for i in range(0,len(same_data)):
    a = random.choice(temp_diff_data)
    final_diff_data_concat.append(a)

same_labels = [1]*len(final_same_data_concat)
diff_labels = [0]*len(final_diff_data_concat)
final_data_set_concat = []
final_data_set_concat.extend(final_same_data_concat)
final_data_set_concat.extend(final_diff_data_concat)
final_train_labels_concat = []
final_train_labels_concat.extend(same_labels)
final_train_labels_concat.extend(diff_labels)
temp = list(zip(final_data_set_concat,final_train_labels_concat))
for i in range(3):
    random.shuffle(temp)
d, t = zip(*temp)

temp_data1 = np.asarray(d)
RawData_gsc_concat = np.transpose(temp_data1)
RawTarget_gsc_concat = list(t)

print(RawData_gsc_concat.shape)
print(len(RawTarget_gsc_concat))

## Create raw data by subtraction

In [ ]:
final_same_data=[]
for i in same_data:
    temp1 = GSC_Dict.get(i[0])
    temp2 = GSC_Dict.get(i[1])
    final_same_data.append(np.absolute(np.asarray(temp1) - np.asarray(temp2)))

temp_diff_data = []
final_diff_data =[]
for i in diff_data:
    temp1 = GSC_Dict.get(i[0])
    temp2 = GSC_Dict.get(i[1]) 
    temp_diff_data.append(np.absolute(np.asarray(temp1) - np.asarray(temp2)))

for i in range(0,len(same_data)):
    a = random.choice(temp_diff_data)
    final_diff_data.append(a)
    
    
same_labels = [1]*len(final_same_data)
diff_labels = [0]*len(final_diff_data)
final_data_set = []
final_data_set.extend(final_same_data)
final_data_set.extend(final_diff_data)
final_train_labels= []
final_train_labels.extend(same_labels)
final_train_labels.extend(diff_labels)
temp = list(zip(final_data_set,final_train_labels))
for i in range(3):
    random.shuffle(temp)
d, t = zip(*temp)

temp_data1 = np.asarray(d)
RawData_gsc_diff = np.transpose(temp_data1)
RawTarget_gsc_diff = list(t)


## Data - Concantenation

## Same Writer

In [ ]:
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget_gsc_concat,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData_gsc_concat,TrainingPercent)

ValDataAct = np.array(GenerateValTargetVector(RawTarget_gsc_concat,ValidationPercent, (len(TrainingTarget))))
ValData    = GenerateValData(RawData_gsc_concat,ValidationPercent, (len(TrainingTarget)))

TestDataAct = np.array(GenerateValTargetVector(RawTarget_gsc_concat,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData_gsc_concat,TestPercent, (len(TrainingTarget)+len(ValDataAct)))

## Gradient Descent for concatenation of features

In [ ]:
gradient_descent(13, RawData_gsc_concat, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 10, 0.17)
gradient_descent(20, RawData_gsc_concat, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 15, 0.20)
gradient_descent(15, RawData_gsc_concat, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 20, 0.23)
gradient_descent(18, RawData_gsc_concat, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 30, 0.25)

## Different Writer

In [ ]:
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget_gsc_diff,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData_gsc_diff,TrainingPercent)

ValDataAct = np.array(GenerateValTargetVector(RawTarget_gsc_diff,ValidationPercent, (len(TrainingTarget))))
ValData    = GenerateValData(RawData_gsc_diff,ValidationPercent, (len(TrainingTarget)))

TestDataAct = np.array(GenerateValTargetVector(RawTarget_gsc_diff,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData_gsc_diff,TestPercent, (len(TrainingTarget)+len(ValDataAct)))

## Gradient Descent for difference of features

In [ ]:
gradient_descent(M, RawData_gsc_diff, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 10, 0.17)
gradient_descent(M, RawData_gsc_diff, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 10, 0.20)
gradient_descent(M, RawData_gsc_diff, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 10, 0.22)
gradient_descent(M, RawData_gsc_diff, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 10, 0.23)
gradient_descent(M, RawData_gsc_diff, TrainingPercent, IsSynthetic, TrainingTarget, C_Lambda, TestData, ValData, 10, 0.24)

## Logistic Regression

In [ ]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    #Sigmoid definition
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    #loss function
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros(X.shape[1])
        
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            
            
            #weight change formula
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient
            
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            loss = self.__loss(h, y)
                
            if(self.verbose ==True and i % 10000 == 0):
                print(f'loss: {loss} \t')
    
    #prediction
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    #round off the probability of the output towards a class
    def predict(self, X):
        return self.predict_prob(X).round()

In [ ]:
X = np.transpose(RawData_concat)
y = np.asarray(RawTarget_concat)
model = LogisticRegression(lr=1.4, num_iter=5000)
model.fit(X, y)
preds = model.predict(X)
(preds == y).mean()


In [ ]:
X = np.transpose(RawData_diff)
y = np.asarray(RawTarget_diff)
model = LogisticRegression(lr=1.4, num_iter=5000)
model.fit(X, y)
preds = model.predict(X)
(preds == y).mean()

In [ ]:
X = np.transpose(RawData_gsc_concat)
y = np.asarray(RawTarget_gsc_concat)
model = LogisticRegression(lr=2.3, num_iter=1000)
model.fit(X, y)
preds = model.predict(X)
(preds == y).mean()

In [ ]:
X = np.transpose(RawData_gsc_diff)
y = np.asarray(RawTarget_gsc_diff)
model = LogisticRegression(lr=0.9, num_iter=1000)
model.fit(X, y)
preds = model.predict(X)
(preds == y).mean()